# Module 2 Spatial Operations

In [ ]:
%matplotlib inline

import pandas as pd
import geopandas

In [ ]:
countries = geopandas.read_file("zip://./data/ne_110m_admin_0_countries.zip")
cities = geopandas.read_file("zip://./data/ne_110m_populated_places.zip")
rivers = geopandas.read_file("zip://./data/ne_50m_rivers_lake_centerlines.zip")

## Point

In [ ]:
paris = cities.loc[cities['name'] == 'Paris', 'geometry'].squeeze()
brussels = cities.loc[cities['name'] == 'Brussels', 'geometry'].squeeze()

In [ ]:
paris

In [ ]:
brussels

## Create Line

In [ ]:
from shapely.geometry import LineString
line = LineString([paris, brussels])

In [ ]:
line

## Polygon

In [ ]:
belgium = countries.loc[countries['name'] == 'Belgium', 'geometry'].squeeze()
belgium

## Combining Shapes

In [ ]:
geopandas.GeoSeries([belgium, paris, brussels, line]).plot(cmap='tab10')

## Spatial Relationship

In [ ]:
brussels.within(belgium)

In [ ]:
belgium.contains(brussels)

In [ ]:
paris.within(belgium)

In [ ]:
belgium.contains(line)

In [ ]:
line.intersects(belgium)

In [ ]:
line.touches(belgium)

## Spatial relationships Boolean

In [ ]:
countries[countries.contains(paris)]

## Ex: Spatial Relationship Boolean

In [ ]:
amazon = rivers[rivers['name'] == 'Amazonas'].geometry.squeeze()

In [ ]:
countries[countries.crosses(amazon)]  # or .intersects

## Set-Operations with Overlay

In [ ]:
from shapely.geometry import Polygon

polys1 = geopandas.GeoSeries([Polygon([(0,0), (2,0), (2,2), (0,2)]),Polygon([(2,2), (4,2), (4,4), (2,4)])])
polys2 = geopandas.GeoSeries([Polygon([(1,1), (3,1), (3,3), (1,3)]),Polygon([(3,3), (5,3), (5,5), (3,5)])])
df1 = geopandas.GeoDataFrame({'geometry': polys1, 'df1':[1,2]})
df2 = geopandas.GeoDataFrame({'geometry': polys2, 'df2':[1,2]})

In [ ]:
ax = df1.plot(color='red');
df2.plot(ax=ax, color='green', alpha=0.5);

In [ ]:
# df3 = geopandas.overlay(df1, df2, how='union')
# df3.plot(color='red',alpha=0.5)
df3= df1.union(df2)
df3.plot(color='red',alpha=0.5)

In [ ]:
#df4 = geopandas.overlay(df1, df2, how='intersection')

df4= df1.intersection(df2)

ax = df3.plot(facecolor='none', edgecolor='k')
df4.plot(ax=ax, color='green',alpha=0.5)

In [ ]:
#df5 = geopandas.overlay(df1, df2, how='difference')


df5= df1.difference(df2)

ax = df3.plot(facecolor='none', edgecolor='k')
df5.plot(ax=ax, color='green',alpha=0.5)

In [ ]:
#df6 = geopandas.overlay(df1, df2, how='symmetric_difference')

df6= df1.symmetric_difference(df2)

ax = df3.plot(facecolor='none', edgecolor='k')
df6.plot(ax=ax, color='green',alpha=0.5)

## Ex: Set-Operations with Overlay

In [ ]:
geopandas.GeoSeries([belgium, brussels.buffer(1)]).plot(alpha=0.5, cmap='tab10')

## Ex2: Set-Operations with Overlay

In [ ]:
africa = countries[countries['continent'] == 'Africa']

africa.plot()

cities['geometry'] = cities.buffer(2)
geopandas.overlay(africa, cities, how='difference').plot()


In [ ]:
brussels.buffer(1).intersection(belgium)

In [ ]:
brussels.buffer(1).union(belgium)

In [ ]:
brussels.buffer(1).difference(belgium)

In [ ]:
brussels.difference(belgium)

## Unary Union

In [ ]:
polys1 = geopandas.GeoSeries([Polygon([(0,0), (2,0), (2,2), (0,2)]),Polygon([(2,2), (4,2), (4,4), (2,4)])])
polys2 = geopandas.GeoSeries([Polygon([(1,1), (3,1), (3,3), (1,3)]),Polygon([(3,3), (5,3), (5,5), (3,5)])])
df1 = geopandas.GeoDataFrame({'geometry': polys1, 'df1':[1,2]})

In [ ]:
df7=df1.unary_union
print(df7)
df7

## Ex: Unary Union

In [ ]:
africa_countries = countries[countries['continent'] == 'Africa']

In [ ]:
africa = africa_countries.unary_union

In [ ]:
africa

## Recap of Pandas DataFrame Join

In [ ]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_a

In [ ]:
raw_data = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_b = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_b

### Inner and Outer Joins

In [ ]:
pd.merge(df_a, df_b, on='subject_id', how='outer')

In [ ]:
pd.merge(df_a, df_b, on='subject_id', how='inner')

In [ ]:
pd.merge(df_a, df_b, on='subject_id')

In [ ]:
cities2 = cities[cities['name'].isin(['Bern', 'Brussels', 'London', 'Paris'])].copy()
cities2['iso_a3'] = ['CHE', 'BEL', 'GBR', 'FRA']

In [ ]:
cities2

In [ ]:
countries2 = countries[['iso_a3', 'name', 'continent']]
countries2.head()

In [ ]:
cities2.merge(countries2, on='iso_a3')

## Spatial Join

In [ ]:
cities_with_country = geopandas.sjoin(cities, countries, how="inner", op='intersects')
cities_with_country

In [ ]:
cities_within_country = geopandas.sjoin(cities, countries, how='left',op='within')

In [ ]:
cities_within_country

In [ ]:
joined['continent'].value_counts()